In [202]:
import numpy as np
import pandas as pd
import pickle
import xgboost
from sklearn.preprocessing import MinMaxScaler

In [203]:
with open('logreg.pkl', 'rb') as fid:
    logreg = pickle.load(fid)
    
with open('randomforest.pkl', 'rb') as fid:
    rf = pickle.load(fid)
    
with open('adaboost.pkl', 'rb') as fid:
    ada = pickle.load(fid)
    
with open('xgboost.pkl', 'rb') as fid:
    xgb = pickle.load(fid)
    
with open('votingclassifier.pkl', 'rb') as fid:
    vclf = pickle.load(fid)
current_stats = pd.read_pickle("current_stats.pkl")
current_stats

,sslpm,slpm_head,slpm_body,slpm_leg,slpm_distance,slpm_clinch,slpm_ground,ssapm,sspercent,kdpm,...,Stance_Sideways,Wins,Losses,kos,subs,decs,lkos,lsubs,ldecs,fighter
0,4.701828,3.416242,0.916991,0.368595,3.506143,0.386575,0.809110,12.397363,0.379260,0.017980,...,0,17,4,3,2,3,2,0,2,Diego Ferreira
1,1.942639,1.317499,0.342819,0.282321,1.250280,0.194936,0.497423,4.550751,0.426883,0.000000,...,0,11,5,0,5,2,1,0,4,Antonio Carlos Junior
2,2.987198,2.005690,0.640114,0.341394,1.408250,0.938834,0.640114,7.766714,0.384615,0.085349,...,0,8,5,0,1,0,1,2,0,Jon Delos Reyes
3,1.733333,1.200000,0.466667,0.066667,0.733333,0.266667,0.733333,3.000000,0.577778,0.066667,...,0,14,3,0,0,1,0,0,0,Pat Sabatini
4,2.638386,1.738231,0.279359,0.620797,2.296948,0.031040,0.310398,6.207967,0.425000,0.000000,...,0,14,4,0,1,0,1,1,1,Vagner Rocha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042,3.295385,1.569231,0.664615,1.061538,2.307692,0.203077,0.784615,6.452308,0.510730,0.027692,...,0,22,10,3,2,1,0,2,4,Kyle Noke
2043,7.555438,5.464625,1.156283,0.934530,5.607181,1.837381,0.110876,18.738120,0.403212,0.031679,...,0,10,7,2,0,2,2,0,1,Leslie Smith
2044,1.723716,0.770171,0.495110,0.458435,1.485330,0.183374,0.055012,5.024450,0.343066,0.036675,...,0,22,11,1,0,0,0,3,2,John Phillips
2045,5.114613,2.514327,1.498926,1.101361,3.835960,0.730659,0.547994,9.691977,0.527716,0.010745,...,0,27,11,1,1,6,1,0,5,Bobby Green


In [204]:
current_stats2 = current_stats.replace([np.inf, -np.inf], np.nan)
current_stats2 = current_stats2.fillna(current_stats2.mean())

In [205]:
current_stats2

,sslpm,slpm_head,slpm_body,slpm_leg,slpm_distance,slpm_clinch,slpm_ground,ssapm,sspercent,kdpm,...,Stance_Sideways,Wins,Losses,kos,subs,decs,lkos,lsubs,ldecs,fighter
0,4.701828,3.416242,0.916991,0.368595,3.506143,0.386575,0.809110,12.397363,0.379260,0.017980,...,0,17,4,3,2,3,2,0,2,Diego Ferreira
1,1.942639,1.317499,0.342819,0.282321,1.250280,0.194936,0.497423,4.550751,0.426883,0.000000,...,0,11,5,0,5,2,1,0,4,Antonio Carlos Junior
2,2.987198,2.005690,0.640114,0.341394,1.408250,0.938834,0.640114,7.766714,0.384615,0.085349,...,0,8,5,0,1,0,1,2,0,Jon Delos Reyes
3,1.733333,1.200000,0.466667,0.066667,0.733333,0.266667,0.733333,3.000000,0.577778,0.066667,...,0,14,3,0,0,1,0,0,0,Pat Sabatini
4,2.638386,1.738231,0.279359,0.620797,2.296948,0.031040,0.310398,6.207967,0.425000,0.000000,...,0,14,4,0,1,0,1,1,1,Vagner Rocha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042,3.295385,1.569231,0.664615,1.061538,2.307692,0.203077,0.784615,6.452308,0.510730,0.027692,...,0,22,10,3,2,1,0,2,4,Kyle Noke
2043,7.555438,5.464625,1.156283,0.934530,5.607181,1.837381,0.110876,18.738120,0.403212,0.031679,...,0,10,7,2,0,2,2,0,1,Leslie Smith
2044,1.723716,0.770171,0.495110,0.458435,1.485330,0.183374,0.055012,5.024450,0.343066,0.036675,...,0,22,11,1,0,0,0,3,2,John Phillips
2045,5.114613,2.514327,1.498926,1.101361,3.835960,0.730659,0.547994,9.691977,0.527716,0.010745,...,0,27,11,1,1,6,1,0,5,Bobby Green


In [206]:
def predict_winner(f1, f2):

    f1stats = np.array(current_stats2[current_stats2["fighter"] == f1])[0][:-1]
    
    f2stats = np.array(current_stats2[current_stats2["fighter"] == f2])[0][:-1]
    
    fight = np.hstack([f1stats, f2stats]).reshape(1, -1)
    
    fight = MinMaxScaler().fit_transform(fight)
    
    result = ada.predict(fight)
    
    if result == 0:
        return f2
    else:
        return f1
    
    

In [207]:
predict_winner("Dustin Poirier", "Nate Diaz")

'Dustin Poirier'